#  Text2SQL Evaluate LLaMA2 Fine-Tune 1 with Exact Match


In [1]:
!pip install -q -U datasets
!pip install -q -U torch auto-gptq transformers optimum
!pip install -q -U peft trl einops accelerate xformers bitsandbytes
#! pip install -q -U rouge_score
#! pip install -q -U langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda

In [2]:
! pip install torchvision==0.20.1+cu124 torch==2.5.1 torchaudio==2.5.1+cu124

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.5/906.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 6.1 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstalled triton-3.2.0
  Attempting uninstall: torch
    Found existing installation: torch 2.6.0
    Uninstalling torch-2.6.0:
      Successfully uninstalled torch-2.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xformers 0.0.29.post2 requires torch==2.6.0, but you have torch 2.5.1 which is incompatible.


In [3]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.33
    Uninstalling langchain-core-0.3.33:
      Successfully uninstalled langchain-core-0.3.33
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.5
    Uninstalling langchain-text-splitters-0.3.5:
      Successfully uninstalled langchain-text-splitters-0.3.5
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.17
    Uninstalling langchain-0.3.17:
      Successfully uninstalled langchain-0.3.17


### Import Necessary Frameworks 

In [4]:
import pandas as pd
import json
import torch
import os
import time
import re
from difflib import SequenceMatcher

# In case Login Required For Model
# from huggingface_hub import login
# from dotenv import load_dotenv

from datasets import load_dataset, Dataset, load_from_disk
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, AutoPeftModelForCausalLM
from transformers import GPTQConfig, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from trl import SFTTrainer
from time import perf_counter
from rich import print

# LangChain Imports
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate  #, LLMChain

# Imports for QA Retrieval Chain
from langchain.chains import RetrievalQA

# Import to Cleanup LLM Output
import textwrap

In [ ]:
# If Login Required for Model Access
# load_dotenv("/notebooks/.env")
# os.environ["TOKENIZERS_PARALLELISM"]="false"
# login(token=os.getenv("HUGGINGFACE_TOKEN"))

#  Let's Define paths for Model and Our dataset 

In [ ]:
model_id = "khalifa1/Text2sql_LLama2_7b_Finetuned"
MODEL_PATH = "/content/drive/MyDrive/Text2SQL/Text2SQL/FTClean/"
MODEL_DIR = "sql_gptq_training"+"_30/"
checkpoint_name ="SQL_llama2_gptq_7b_peftv1_20230930_043049"

In [6]:
print(checkpoint_name)

SQL_llama2_gptq_7b_peftv1_20230930_043049

In [17]:
# GDrive Location for Train/Test Data
DATA_PATH ="/content/drive/MyDrive/Text2SQL/Text2SQL/Data/"
DS_DIR = "sql_train_test"
PKL_DIR = "test/"
PKL_FILE ="sql_test.pkl"
LLMRES_PKL_FILE="sql_test_llmres_from300_to_400.pkl"

## Let's Define some Helping Functions

In [8]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

In [9]:
def parse(text):
    start_marker = '### Response:'
    end_marker = '### End'
    start_index = text.find(start_marker)
    end_index = text.find(end_marker, start_index + len(start_marker))

    return (text[start_index + len(start_marker):].strip() if start_index != -1 and end_index == -1
            else text[start_index + len(start_marker):end_index].strip() if start_index != -1
            else None)

In [10]:
def gen_eval_res(test_df):
  eval_res = []
  time_res = []
  row_no = 1
  # tmp = test_df.iloc[:2]
  for row in test_df.itertuples(index=False):
    start_time = perf_counter()
    # print(row.text)
    text = row.text # test_df['text'][ID]
    llm_res = pipe(text)[0]['generated_text']
    parse_res = parse(llm_res)
    end_time = perf_counter()
    eval_res.append(parse_res)
    time_res.append(end_time-start_time)
    print(f"*** Row {row_no} Done ***")
    row_no += 1

  test_df['eval_res'] = eval_res
  test_df['eval_time'] = time_res
  return test_df
  # display(pd.DataFrame(eval_res))
  # display(pd.DataFrame(time_res))

# let's now Load and Check Data From drive

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load Test & Training Data from Disk
dataset = load_from_disk(DATA_PATH + DS_DIR)

In [13]:
test_df = pd.read_pickle(DATA_PATH + PKL_DIR + PKL_FILE)

In [14]:
display(dataset['train'])
display(dataset['test'])

Dataset({
    features: ['response', 'question', 'context', 'text', '__index_level_0__'],
    num_rows: 4086
})

Dataset({
    features: ['response', 'question', 'context', 'text', '__index_level_0__'],
    num_rows: 454
})

In [15]:
display(test_df.head(2))
display(test_df.shape)

,response,question,context,__index_level_0__,text
0,SELECT College FROM match_season GROUP BY Coll...,Show the name of colleges that have at least t...,CREATE TABLE match_season (College VARCHAR),611,### Instruction:\n You are a powerful text-...
1,SELECT T1.name FROM Person AS T1 JOIN PersonFr...,Find the name of the person who has friends wi...,"CREATE TABLE Person (name VARCHAR, age INTEGER...",2556,### Instruction:\n You are a powerful text-...


(454, 5)

### Load Model and it 's Tokenizer 

In [16]:
output_dir = MODEL_PATH + MODEL_DIR + checkpoint_name
print(output_dir)

/content/drive/MyDrive/Text2SQL/Text2SQL/FTClean/sql_gptq_training_30/SQL_llama2_gptq_7b_peftv1_20230930_043049

In [ ]:


# Tokenizer - Base Model since we did not change it
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# LLM GPTQ Fine-Tune Model
# To perform inference on the test dataset
model = AutoPeftModelForCausalLM.from_pretrained(
          output_dir,
          low_cpu_mem_usage=True,
          return_dict=True,
          torch_dtype=torch.float16,
          device_map="cuda", # Forces model to use GPU
      )

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=180,
    do_sample = True,
    temperature=0.1,
    top_k=4,
    # top_p=0.95,
    repetition_penalty=1.2,
    use_cache=True
)
#  penalty_alpha=0.5,
#     # do_sample = True,
#     top_k=1,
#     # temperature=0.1,
#     repetition_penalty=1.2,
#     max_new_tokens=180

llm = HuggingFacePipeline(pipeline=pipe)

#### Check Model on Sample Test Question

In [26]:
ID = 12

In [23]:
test_df['text'][ID]

'### Instruction:\n    You are a powerful text-to-SQL model.     Your job is to answer questions about a database.     You are given a question and context regarding one or more tables.\n\n    You must output the SQL query that answers the question.\n\n    ### Input:\n    What is the platform if the weight is 131.5g?\n    ### Context:\n    CREATE TABLE table (platform VARCHAR, weight VARCHAR)\n    ### Response:\n    '

In [27]:
display(test_df['response'][ID])

'SELECT Status FROM roller_coaster GROUP BY Status HAVING COUNT(*) > 2'

In [28]:
text = test_df['text'][ID]
start_time = perf_counter()
llm_res = pipe(text)[0]['generated_text']
parse_res = parse(llm_res)
end_time = perf_counter()
print(f"Time Taken: {end_time-start_time}")
# print(llm_res)
print(parse_res)

Time Taken: 20.993395767000038

SELECT Status FROM roller_coaster GROUP BY Status HAVING COUNT(*) > 2

### Get LLM Responses For All Questions

In [ ]:
# def gen_eval_res(test_df):
#   eval_res = []
#   time_res = []
#   # tmp = test_df.iloc[:2]
#   for row in test_df.itertuples(index=False):
#     start_time = perf_counter()
#     # print(row.text)
#     text = row.text # test_df['text'][ID]
#     llm_res = pipe(text)[0]['generated_text']
#     parse_res = parse(llm_res)
#     end_time = perf_counter()
#     eval_res.append(parse_res)
#     time_res.append(end_time-start_time)

#   test_df['eval_res'] = eval_res
#   test_df['eval_time'] = time_res
#   # display(pd.DataFrame(eval_res))
#   # display(pd.DataFrame(time_res))

In [29]:
eval_start_time = perf_counter()
test_df=gen_eval_res(test_df.iloc[300:400]) # test_df.iloc[:4]
eval_end_time = perf_counter()
print('Evaluation Took: ', round(eval_end_time-eval_start_time, 2))

*** Row 1 Done ***

*** Row 2 Done ***

*** Row 3 Done ***

*** Row 4 Done ***

*** Row 5 Done ***

...

*** Row 98 Done ***

*** Row 99 Done ***

*** Row 100 Done ***

<ipython-input-10-f9dd0a9fc6b2>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['eval_res'] = eval_res
<ipython-input-10-f9dd0a9fc6b2>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['eval_time'] = time_res


Evaluation Took:  4456.15

In [30]:
display(test_df.head(10))

,response,question,context,__index_level_0__,text,eval_res,eval_time
300,"SELECT directed_by, COUNT(*) FROM film GROUP B...",List all directors along with the number of fi...,CREATE TABLE film (directed_by VARCHAR),1149,### Instruction:\n You are a powerful text-...,"SELECT COUNT(*), DIRECTED_BY FROM film GROUP B...",31.235987
301,"SELECT T2.allergytype, COUNT(*) FROM Has_aller...",Show all allergy type with number of students ...,CREATE TABLE Has_allergy (allergy VARCHAR); CR...,290,### Instruction:\n You are a powerful text-...,SELECT T2.allergypeffected FROM Allergy_type A...,61.443958
302,SELECT Advisor FROM Student WHERE StuID = 1004,Who is the advisor of student with ID 1004?,"CREATE TABLE Student (Advisor VARCHAR, StuID V...",271,### Instruction:\n You are a powerful text-...,SELECT Advisor FROM Student WHERE StuID = 1004,20.971152
303,SELECT Name FROM singer ORDER BY Net_Worth_Mil...,List the name of singers in ascending order of...,"CREATE TABLE singer (Name VARCHAR, Net_Worth_M...",4501,### Instruction:\n You are a powerful text-...,SELECT Name FROM singer ORDER BY Net_Worth_Mil...,17.729207
304,SELECT T2.country FROM city AS T1 JOIN country...,Which countries have at least 3 cities?,"CREATE TABLE country (country VARCHAR, country...",1741,### Instruction:\n You are a powerful text-...,SELECT T1.country FROM country AS T1 JOIN city...,41.285053
305,SELECT COUNT(*) FROM Video_games,How many video games exist?,CREATE TABLE Video_games (Id VARCHAR),3405,### Instruction:\n You are a powerful text-...,SELECT COUNT(*) FROM Video_games,12.971104
306,SELECT catalog_entry_name FROM catalog_content...,What is the product with the highest height? G...,CREATE TABLE catalog_contents (catalog_entry_n...,193,### Instruction:\n You are a powerful text-...,SELECT T1.catalog_entry_name FROM catalog_cont...,31.533715
307,"SELECT billing_country, AVG(total) FROM invoic...",A list of the top 10 countries by average invo...,CREATE TABLE invoices (billing_country VARCHAR...,302,### Instruction:\n You are a powerful text-...,"SELECT billing_country, AVG(total) FROM invoic...",36.332352
308,SELECT DISTINCT T2.pilot FROM airport AS T1 JO...,What are the different pilot names who had pil...,"CREATE TABLE airport (id VARCHAR, country VARC...",3625,### Instruction:\n You are a powerful text-...,SELECT DISTINCT T1.name FROM airport AS T2 JOI...,53.393605
309,SELECT COUNT(*) FROM Player WHERE HS > 1000,How many players have more than 1000 hours of ...,CREATE TABLE Player (HS INTEGER),2848,### Instruction:\n You are a powerful text-...,SELECT COUNT(*) FROM Player WHERE HS > 1000,17.787914


In [31]:
test_df.to_pickle(DATA_PATH + PKL_DIR + LLMRES_PKL_FILE)

In [ ]:
# display(test_df['response'][:4].values)

In [8]:
test_df2=pd.read_pickle(DATA_PATH + PKL_DIR + LLMRES_PKL_FILE)
display(test_df2.head(2))

,response,question,context,__index_level_0__,text,eval_res,eval_time
300,"SELECT directed_by, COUNT(*) FROM film GROUP B...",List all directors along with the number of fi...,CREATE TABLE film (directed_by VARCHAR),1149,### Instruction:\n You are a powerful text-...,"SELECT COUNT(*), DIRECTED_BY FROM film GROUP B...",31.235987
301,"SELECT T2.allergytype, COUNT(*) FROM Has_aller...",Show all allergy type with number of students ...,CREATE TABLE Has_allergy (allergy VARCHAR); CR...,290,### Instruction:\n You are a powerful text-...,SELECT T2.allergypeffected FROM Allergy_type A...,61.443958


In [20]:
LLMRES_PKL_FILE1="sql_test_llmres.pkl"
LLMRES_PKL_FILE2="sql_test_llmres_v2.pkl"
LLMRES_PKL_FILE3="sql_test_llmres_from200_to_300.pkl"
LLMRES_PKL_FILE4="sql_test_llmres_from300_to_400.pkl"

In [21]:
import pandas as pd
fisrt_res =pd.read_pickle(DATA_PATH + PKL_DIR + LLMRES_PKL_FILE1)
second_res =pd.read_pickle(DATA_PATH + PKL_DIR + LLMRES_PKL_FILE2)
third_res =pd.read_pickle(DATA_PATH + PKL_DIR + LLMRES_PKL_FILE3)
fourth_res =pd.read_pickle(DATA_PATH + PKL_DIR + LLMRES_PKL_FILE4)

In [26]:
final_LLM_res=pd.concat([fisrt_res, second_res, third_res, fourth_res],ignore_index=True)


In [27]:
final_LLM_res

,response,question,context,__index_level_0__,text,eval_res,eval_time
0,SELECT College FROM match_season GROUP BY Coll...,Show the name of colleges that have at least t...,CREATE TABLE match_season (College VARCHAR),611,### Instruction:\n You are a powerful text-...,SELECT College FROM match_season GROUP BY Coll...,23.327458
1,SELECT T1.name FROM Person AS T1 JOIN PersonFr...,Find the name of the person who has friends wi...,"CREATE TABLE Person (name VARCHAR, age INTEGER...",2556,### Instruction:\n You are a powerful text-...,SELECT T1.name FROM Person AS T1 JOIN PersonFr...,54.584005
2,"SELECT AVG(T1.price), T2.name FROM products AS...",Find the average prices of all products from e...,"CREATE TABLE products (price INTEGER, Manufact...",3026,### Instruction:\n You are a powerful text-...,"SELECT AVG(T1.Price), T2.Name FROM products AS...",147.370144
3,"SELECT Support_rate, Consider_rate, Oppose_rat...","Please list support, consider, and oppose rate...","CREATE TABLE candidate (Support_rate VARCHAR, ...",1438,### Instruction:\n You are a powerful text-...,"SELECT Support_rate, Consider_rate, Oppose_rat...",28.611904
4,"SELECT Fname, Lname FROM Student WHERE Age > 1...",List all female students age is older than 18 ...,"CREATE TABLE Student (Fname VARCHAR, Lname VAR...",1676,### Instruction:\n You are a powerful text-...,"SELECT Fname, Lname FROM Student WHERE Sex = ""...",69.530650
...,...,...,...,...,...,...,...
396,"SELECT first_name, last_name, email_address FR...",Which owners live in the state whose name cont...,"CREATE TABLE Owners (first_name VARCHAR, last_...",4486,### Instruction:\n You are a powerful text-...,"SELECT T1.first_name, T1.last_name, T2.email F...",53.536297
397,SELECT COUNT(*) FROM FLIGHTS AS T1 JOIN AIRLIN...,How many flights does airline 'JetBlue Airways...,CREATE TABLE FLIGHTS (Airline VARCHAR); CREATE...,4077,### Instruction:\n You are a powerful text-...,SELECT COUNT(*) FROM FLIGHTS AS T1 JOIN AIRLIN...,45.360868
398,SELECT DISTINCT Country FROM manager,Show the distinct countries of managers.,CREATE TABLE manager (Country VARCHAR),3218,### Instruction:\n You are a powerful text-...,SELECT DISTINCT Country FROM manager,15.385629
399,"SELECT email_address, phone_number FROM custom...",Find the emails and phone numbers of all the c...,"CREATE TABLE customers (email_address VARCHAR,...",3285,### Instruction:\n You are a powerful text-...,"SELECT email_address, phone_number FROM custom...",27.497009


In [28]:
final_LLM_res.to_pickle(DATA_PATH + PKL_DIR + "sql_test_llmres_final.pkl")

In [29]:
final_LLM_res=pd.read_pickle(DATA_PATH + PKL_DIR + "sql_test_llmres_final.pkl")
final_LLM_res.head(5)

,response,question,context,__index_level_0__,text,eval_res,eval_time
0,SELECT College FROM match_season GROUP BY Coll...,Show the name of colleges that have at least t...,CREATE TABLE match_season (College VARCHAR),611,### Instruction:\n You are a powerful text-...,SELECT College FROM match_season GROUP BY Coll...,23.327458
1,SELECT T1.name FROM Person AS T1 JOIN PersonFr...,Find the name of the person who has friends wi...,"CREATE TABLE Person (name VARCHAR, age INTEGER...",2556,### Instruction:\n You are a powerful text-...,SELECT T1.name FROM Person AS T1 JOIN PersonFr...,54.584005
2,"SELECT AVG(T1.price), T2.name FROM products AS...",Find the average prices of all products from e...,"CREATE TABLE products (price INTEGER, Manufact...",3026,### Instruction:\n You are a powerful text-...,"SELECT AVG(T1.Price), T2.Name FROM products AS...",147.370144
3,"SELECT Support_rate, Consider_rate, Oppose_rat...","Please list support, consider, and oppose rate...","CREATE TABLE candidate (Support_rate VARCHAR, ...",1438,### Instruction:\n You are a powerful text-...,"SELECT Support_rate, Consider_rate, Oppose_rat...",28.611904
4,"SELECT Fname, Lname FROM Student WHERE Age > 1...",List all female students age is older than 18 ...,"CREATE TABLE Student (Fname VARCHAR, Lname VAR...",1676,### Instruction:\n You are a powerful text-...,"SELECT Fname, Lname FROM Student WHERE Sex = ""...",69.530650


## finally Let's Calculate Exact Match Accurcy for our finetune Model LLama 2 7B

In [ ]:
import re
from difflib import SequenceMatcher
def normalize_text(text):
    """Remove punctuation, convert to lowercase, and strip extra spaces."""
    return re.sub(r'\W+', ' ', text).strip().lower()

def calculate_exact_match(df, threshold=0.90):
    def is_match(response, eval_res):
        normalized_response = normalize_text(response)
        normalized_eval = normalize_text(eval_res)
        similarity = SequenceMatcher(None, normalized_response, normalized_eval).ratio()
        return similarity >= threshold

    df['exact_match'] = df.apply(lambda row: is_match(row['response'], row['eval_res']), axis=1)
    accuracy = df['exact_match'].sum() / len(df)
    mismatches = df[~df['exact_match']]
    print(f"Exact Match Accuracy: {accuracy:.2%}")
    print(f"\nSample Mismatches ({len(mismatches)} cases):")

    return mismatches.sample(min(3, len(mismatches)))



In [35]:
mismatches = calculate_exact_match(final_LLM_res)
display(mismatches)

Exact Match Accuracy: 63.59%

Sample Mismatches (146 cases):

,response,question,context,__index_level_0__,text,eval_res,eval_time,exact_match
135,SELECT COUNT(DISTINCT s_id) FROM advisor,How many students have advisors?,CREATE TABLE advisor (s_id VARCHAR),774,### Instruction:\n You are a powerful text-...,SELECT COUNT(*) FROM student AS T1 JOIN adviso...,44.927328,False
1,SELECT T1.name FROM Person AS T1 JOIN PersonFr...,Find the name of the person who has friends wi...,"CREATE TABLE Person (name VARCHAR, age INTEGER...",2556,### Instruction:\n You are a powerful text-...,SELECT T1.name FROM Person AS T1 JOIN PersonFr...,54.584005,False
151,"SELECT T2.customer_name, T3.city, T1.date_from...","Show the customer name, customer address city,...","CREATE TABLE addresses (city VARCHAR, address_...",3232,### Instruction:\n You are a powerful text-...,"SELECT T1.customer_name, T2.city, MIN(T3.date_...",144.539996,False
